In [22]:
import cv2

In [23]:
path = '/disk2/workspace/csgoai/stone/csgo_ai/sample_data/data/roth-1679837416-919064000/1679839124-203183400.jpg'
!ls /disk2/workspace/csgoai/stone/csgo_ai/sample_data/data/roth-1679837416-919064000/1679839124-203183400.jpg

/disk2/workspace/csgoai/stone/csgo_ai/sample_data/data/roth-1679837416-919064000/1679839124-203183400.jpg


In [24]:
img = cv2.imread(path)
%timeit imgs = [cv2.resize(img,(768, 1024)) for each in range(180)]

124 ms ± 486 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [32]:
import os
base = '/disk3/csgo_bak/csgo_ai_data/data/roth-1679837416-919064000/'
names = os.listdir(base)
names = [base+each for each in names][:180]

In [33]:
%timeit imgs = [cv2.imread(each) for each in names]

3.46 s ± 1.18 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [41]:
base = '/disk3/tt/'
names = os.listdir(base)
names = [base+each for each in names][:180]
%timeit imgs = [cv2.imread(each) for each in names]

3.23 s ± 2.58 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [42]:
base2 = '/home/ai/stone/test/roth-1679837416-919064000/'
names = os.listdir(base2)
names = [base2+each for each in names][:180]
%timeit imgs = [cv2.imread(each) for each in names]

3.25 s ± 3.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
from concurrent.futures import ProcessPoolExecutor
import numpy as np
import cv2

def load_images(img_paths):
    img_array = np.zeros((len(img_paths),3,150,280), dtype=np.uint8)
    for i, img_path in enumerate(img_paths):
        img_array[i, :, :, :] = img_preprocess(cv2.resize(cv2.imread(img_path), (768, 1024)))
    return img_array

def parallel_load_images(img_paths, num_workers=4):
    img_chunks = np.array_split(img_paths, num_workers)
    results = None
    
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        futures = [executor.submit(load_images, chunk) for chunk in img_chunks]
        results = [f.result() for f in futures]

    return np.concatenate(results, axis=0)

img_paths = [...] # 图像路径列表
img_array = parallel_load_images(img_paths)
@顺序不变 可以多进程加载图像和预处理

In [44]:
import cv2
import os
import time

start_time = time.time()

# 设置文件夹路径
folder_path = base2#'C:/your/folder/path/'

# 遍历文件夹并读取图像
for filename in os.listdir(folder_path)[:180]:
    if filename.endswith('.jpg'):
        filepath = os.path.join(folder_path, filename)
        img = cv2.imread(filepath)
        # 对图像进行处理
        # ...

end_time = time.time()
total_time = end_time - start_time
print('总时间：', total_time, 's')

总时间： 3.1849722862243652 s


In [48]:
import cv2
import os
import time
from multiprocessing import Pool, cpu_count

# 设置文件夹路径
folder_path = base2#'C:/your/folder/path/'
num_processes = cpu_count()  # 获取CPU核心数
print(num_processes)

def process_image(filename):
    if filename.endswith('.jpg'):
        filepath = os.path.join(folder_path, filename)
        img = cv2.imread(filepath)
        # 对图像进行处理
        # ...

start_time = time.time()

# 创建进程池并以并行方式处理所有图像
with Pool(num_processes) as p:
    p.map(process_image, os.listdir(folder_path)[:180])

end_time = time.time()
total_time = end_time - start_time
print('总时间：', total_time, 's')


40
总时间： 0.9338948726654053 s


In [49]:
import os
import time
from concurrent.futures import ProcessPoolExecutor
import cv2

# 设置路径
folder_path = base2#'C:/your/folder/path/'

def process_image(filename):
    if filename.endswith('.jpg'):
        filepath = os.path.join(folder_path, filename)
        img = cv2.imread(filepath)
        # 对图像进行处理
        # ...

start_time = time.time()

with ProcessPoolExecutor() as executor:
    executor.map(process_image, os.listdir(folder_path)[:180])

end_time = time.time()
total_time = end_time - start_time

print(f"总时间：{total_time:.2f}秒")


总时间：0.80秒


In [50]:
import cv2
import os
import time
from concurrent.futures import ProcessPoolExecutor, as_completed
import numpy as np
import multiprocessing.shared_memory as shm

# 设置文件夹路径和读取未压缩图像
folder_path = base2#'C:/your/folder/path/'
read_mode = cv2.IMREAD_UNCHANGED

# 加载所有图像到内存中
imgs_buffer = []
for filename in os.listdir(folder_path)[:180]:
    if not filename.endswith('.jpg'):
        continue
    filepath = os.path.join(folder_path, filename)
    img = cv2.imread(filepath, read_mode)
    imgs_buffer.append(img)

# 将图像数据共享给各个进程

shms = []
for i, img in enumerate(imgs_buffer):
    shm_name = f"img_{i}"
    shm_img = shm.SharedMemory(create=True, size=img.nbytes, name=shm_name)
    shms.append(shm_img)
    np_img = np.ndarray(img.shape, dtype=img.dtype, buffer=shm_img.buf)
    np_img[:] = img[:]
    shm.SharedMemory(close_fd=True, name=shm_name)

# 处理图像的函数
def process_image(idx):
    shm_name = f"img_{idx}"
    shm_img = shm.SharedMemory(name=shm_name)
    img = np.ndarray(shape=imgs_buffer[idx].shape, dtype=imgs_buffer[idx].dtype,
                     buffer=shm_img.buf)
    # 对图像进行处理
    # ...



start_time = time.time()

with ProcessPoolExecutor() as executor:
    futures = [executor.submit(process_image, i) for i in range(len(imgs_buffer))]

    # 等待所有任务完成
    for future in as_completed(futures):
        if future.exception() is not None:
            print(f"Exception: {future.exception()}")

end_time = time.time()
total_time = end_time - start_time

print(f"总时间：{total_time:.2f}秒")

for shm_img in shms:
    shm_img.close()
    shm_img.unlink()


TypeError: __init__() got an unexpected keyword argument 'close_fd'

In [7]:
import numpy as np
a = np.zeros((10000,3,1024,768), dtype=np.uint8)
a.nbytes / 1024 / 1024
#4*1024*768*3*1100/1024/1024

22500.0

In [ ]:
# 读取数据
class MyDataset(data.Dataset):
    def __init__(self, archive,image='image',mask='mask'):
        self.archive = h5py.File(archive, 'r')
        self.data = self.archive[image]
        self.labels = self.archive[mask]

    def __getitem__(self, index):
        image = self.data[index]
        mask = self.get_multi_class_labels(self.labels[index])
        return image, mask

    def __len__(self):
        return len(self.labels)

    def get_multi_class_labels(self,truth, n_labels=3, labels=(0, 1, 2)):
        new_shape =  [n_labels, ]+list(truth.shape[1:])
        y = np.zeros(new_shape, np.int8)
        for label_index in range(n_labels):
            if labels is not None:
                y[label_index, :, :][truth[0, :, :] == labels[label_index]] = 1
            else:
                y[label_index, :, :][truth[0, :, :] == label_index] = 1
        return y

    def close(self):
        self.archive.close()
